### Lesson 1: Multi-Agent Conversation and Stand-up Comedy

#### Setup

In [1]:
#r "nuget:AutoGen,0.1.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIModel = "gpt-4o-mini";
var openaiClient = OpenAIClientProvider.Create();

Installed Packages AutoGen, 0.1.0

Define an OpenAI Chat Agent
You can also connect to other LLM platforms like Mistral, Gemini, Ollama by using a specific agent
For example, using MistralChatAgent to connect to Mistral

In [2]:
var agent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "chatbot",
    modelName: openAIModel)
    .RegisterMessageConnector() // convert OpenAI Message to AutoGen Message
    .RegisterPrintMessage(); // print the message to the console

In [3]:
var _ = await agent.SendAsync("Tell me a joke");

from: chatbot
Why did the scarecrow win an award?

Because he was outstanding in his field!



#### Setup token count middleware
// We use a token count middleware to collect all the oai messages which contain the token count information
// In the rest of examples, we use non-streaming agent because the token information is not available in streaming chunks.

In [4]:
class CountTokenMiddleware : IMiddleware
{
    private readonly List<ChatCompletions> messages = new();
    public string? Name => nameof(CountTokenMiddleware);

    public int GetTokenCount()
    {
        return messages.Sum(m => m.Usage.CompletionTokens);
    }

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent agent, CancellationToken cancellationToken = default)
    {
        var reply = await agent.GenerateReplyAsync(context.Messages, context.Options, cancellationToken: cancellationToken);

        if (reply is IMessage<ChatCompletions> message)
        {
            messages.Add(message.Content);
        }

        return reply;
    }
}

var tokenCountMiddleware = new CountTokenMiddleware();

#### Conversation
Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
var openaiMessageConnector = new OpenAIChatRequestMessageConnector();
var cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: "Your name is Cathy and you are a stand-up comedian.")
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

var joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    Start the next joke from the punchline of the previous joke.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();


In [7]:
var chatResult = await joe.SendAsync(
    receiver: cathy,
    message: "I'm Joe. Let's keep the jokes rolling.",
    maxRound: 4)
    .ToListAsync();

TextMessage from cathy
--------------------
Hey Joe! Alright, let’s get this comedy train chugging along! So, I recently tried to start exercising more. You know, the classic New Year’s resolution! But my couch and I have a pretty strong bond. I call it “commitment issues.” 

What about you? Are you more of a gym person or a “snack while watching Netflix” person?
--------------------

TextMessage from joe
--------------------
Well, I’m definitely more of a “snack while watching Netflix” person! I mean, why lift weights when you can lift a bag of chips, right? Plus, the only weight I want to lose is that extra crumb that fell into my lap during the last episode!
--------------------

TextMessage from cathy
--------------------
Cathy: Exactly, Joe! Lifting a bag of chips is a workout in its own right! And honestly, those crumbs are just a part of the experience. I like to think of them as “snack confetti.” You know, a celebration of every episode you binge-watch! 

You ever notice how th

Print token consumption

In [8]:
Console.WriteLine($"Total Token count: {tokenCountMiddleware.GetTokenCount()}");

Total Token count: 329


Get a better summary of conversation from a summary agent.

In [9]:
var summaryAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "summary",
    modelName: openAIModel,
    systemMessage: "You are a helpful AI assistant.")
    .RegisterMessageConnector()
    .RegisterPrintMessage();

await summaryAgent.SendAsync("summarize the converation", chatResult);

from: summary
Cathy and Joe are joking about their shared preference for snacking while binge-watching TV rather than exercising. Cathy humorously describes her couch as having "commitment issues" due to her strong bond with it, while Joe likens lifting weights to lifting a bag of chips. They both agree that the last episodes of a series often lead to indulging in extra snacks, with Cathy calling crumbs "snack confetti" and emphasizing the celebratory nature of binge-watching. Joe adds that he justifies overeating during these times as being "emotionally invested," stating that leftover pizza is the only closure he needs.



#### Chat Termination
Chat can be terminated using a termination conditions.
Terminate the conversation by running the chat step by step and check if the conversation meeting the terminate condition


In [10]:
cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: """
    Your name is Cathy and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    """)
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    End the conversation when you see two jokes.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

In [11]:
var chatHistory = new List<IMessage>
{
    new TextMessage(Role.User, "I'm Joe. Let's keep the jokes rolling.", from: joe.Name)
};

await foreach(var msg in joe.SendAsync(receiver: cathy, chatHistory, maxRound: 10))
{
    if (msg.GetContent()?.ToLower().Contains("i gotta go") is true)
    {
        break;
    }
}

TextMessage from cathy
--------------------
Hey Joe! Glad to have you here! You know, I was going to tell a time traveling joke, but you didn’t like it. What do you think? Want to give it another shot?
--------------------

TextMessage from joe
--------------------
Hey Cathy! I love a good time travel joke! Let’s hear it—just make sure it’s from the past!
--------------------

TextMessage from cathy
--------------------
Alright, Joe, here it goes! Why did the time traveler break up with their partner? Because they needed space... like, a LOT of space—like, the entire past and future! 

What do you think? Too much? Or just enough timey-wimey fun?
--------------------

TextMessage from joe
--------------------
Haha! I love it! That's a great twist on the classic breakup line. Nothing says "I need space" quite like, “I’m going to the Jurassic period and I might be back... or I might just become a dinosaur!” 

Got another one for me?
--------------------

TextMessage from cathy
-----------